In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix

# Charger les données
metadata = pd.read_csv("articles_metadata.csv")
clicks = pd.concat(
    [pd.read_csv(os.path.join("clicks", f)) for f in os.listdir("clicks")],
    ignore_index=True
)

# Encodage users & articles
user_ids = clicks["user_id"].astype("category")
article_ids = clicks["click_article_id"].astype("category")

user_map = dict(enumerate(user_ids.cat.categories))       # index → user_id
article_map = dict(enumerate(article_ids.cat.categories)) # index → article_id

# Construction matrice utilisateur-article (interactions binaires)
matrix = coo_matrix(
    (np.ones(len(clicks)), (user_ids.cat.codes, article_ids.cat.codes))
).tocsr()

# Entraînement du modèle ALS (implicit feedback)
model = AlternatingLeastSquares(
    factors=64,      # dimension des embeddings
    regularization=0.01,
    iterations=20,
    use_gpu=False    # True si tu as CUDA
)
model.fit(matrix)

# Sauvegarder le modèle + mappings
with open("implicit_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("user_map.pkl", "wb") as f:
    pickle.dump(user_map, f)

with open("article_map.pkl", "wb") as f:
    pickle.dump(article_map, f)

print("✅ Modèle ALS entraîné et sauvegardé !")


C:\Users\elect\anaconda3\envs\surprise_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\elect\anaconda3\envs\surprise_env\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 24 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 20/20 [00:08<00:00,  2.42it/s]

✅ Modèle ALS entraîné et sauvegardé !
